# TP6 : Diffusion


Challenge: make the following code work. 


In [7]:
import numpy as np
import random
import os
import torch
import torchvision

from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import math

import torch.nn.functional as F

from torch.utils.data import DataLoader
import torchvision.utils as utils
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("ebrahimelgazar/pixel-art")

# print("Path to dataset files:", path)

# Define the diffusion model

We structure the code as follows:
- A `Diffusion` class that handles the diffusion process, including adding noise and sampling.
- A simple neural network model to predict the noise added at each step.

The diffusion class implement the forward and reverse diffusion processes. 

The forward process adds noise to an image to create the noisy image at time step t.

The reverse proces takes a model an image and a time step and predicts the denoised image at the previous time step.

In [ ]:
# Define the diffusion interface
class Diffusion:
    def __init__(self, T=100, beta_start=1e-4, beta_end=2e-2):
        self.T = T
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.betas = torch.linspace(beta_start, beta_end, T)  # T,
        self.alphas = 1 - self.betas  # T
        self.a_bar = torch.cumprod(self.alphas, 0)

    def q_sample(self, x0, t, noise=None):
        noise = torch.randn_like(x0)  # b,1,28,28
        a_bar_t = self.a_bar[t].view(-1, 1, 1, 1).to(x0.device)  # b,1,1,1
        return (a_bar_t.sqrt() * x0 + (1 - a_bar_t).sqrt() * noise), noise

    @torch.no_grad()
    def p_sample(self, model, x_t, t):
        b = x_t.size(0)
        # self.betas[t]  # b,
        betat = self.betas[t].view(b, 1, 1, 1).to(x_t.device)
        a_t = self.alphas[t].view(b, 1, 1, 1).to(x_t.device)
        a_bar_t = self.a_bar[t].view(b, 1, 1, 1).to(x_t.device)
        eps = model(x_t, t)
        mean = (1 / a_t.sqrt()) * (x_t - (betat / (1 - a_bar_t).sqrt()) * eps)
        if (t > 0).any():
            z = torch.randn_like(x_t)
            return mean + betat.sqrt() * z
        return mean

    @torch.no_grad()
    def sample(self, model, shape, device):
        x = torch.randn(shape, device=device)
        for t in reversed(range(self.T)):
            x = self.p_sample(
                model, x, torch.full((shape[0],), t, device=device, dtype=torch.long)
            )
        return x

In [14]:
def seed_all(seed=0):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# timestep embedding
def timestep_embedding(t, dim):
    # produces sinusoidal embeddings for timesteps
    half = dim // 2
    freqs = torch.exp(-math.log(10000) * torch.arange(0, half, device=t.device) / half)
    args = t[:, None].float() * freqs[None]
    emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)
    if dim % 2 == 1:
        emb = F.pad(emb, (0, 1))
    return emb


# ---------------- Model ----------------
class TinyUNet(nn.Module):
    def __init__(self, base=32):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.Linear(64, base), nn.SiLU(), nn.Linear(base, base)
        )
        self.down1 = nn.Conv2d(1, base, 3, padding=1)
        self.down2 = nn.Conv2d(base, base * 2, 4, 2, 1)
        self.mid = nn.Conv2d(base * 2, base * 2, 3, padding=1)
        self.up1 = nn.ConvTranspose2d(base * 2, base, 4, 2, 1)
        self.out = nn.Conv2d(base, 1, 3, padding=1)

        self.time_emb2 = nn.Conv2d(base, base * 2, 1)
        self.time_emb_mid = nn.Conv2d(base, base * 2, 1)
        self.time_emb_up = nn.Conv2d(base, base, 1)

    def forward(self, x, t):
        t_emb = self.time_mlp(timestep_embedding(t, 64)).unsqueeze(-1).unsqueeze(-1)

        h1 = F.silu(self.down1(x) + t_emb)
        h2 = F.silu(self.down2(h1) + self.time_emb_mid(t_emb))
        h = F.silu(self.mid(h2) + self.time_emb_mid(t_emb))
        h = F.silu(self.up1(h + h2) + self.time_emb_up(t_emb))
        out = self.out(h + h1)
        return out


class SimpleConvNet(nn.Module):
    def __init__(self, channels=1, base=64, time_emb_dim=64):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.Linear(time_emb_dim, base), nn.GELU(), nn.Linear(base, base)
        )

        # simple stack of conv layers
        self.conv1 = nn.Conv2d(channels, base, 3, padding=1)
        self.conv2 = nn.Conv2d(base, base, 3, padding=1)
        # self.conv3 = nn.Conv2d(base, base, 3, padding=1)
        self.conv_out = nn.Conv2d(base, channels, 3, padding=1)

        self.act = nn.GELU()
        self.norm = nn.GroupNorm(8, base)

    def forward(self, x, t):
        t_emb = self.time_mlp(timestep_embedding(t, 64)).unsqueeze(-1).unsqueeze(-1)

        h = self.conv1(x)
        h = self.norm(self.act(h + t_emb))
        h = self.conv2(h)
        h = self.norm(self.act(h + t_emb))
        # h = self.conv3(h)
        # h = self.norm(self.act(h + t_emb))
        return self.conv_out(h)


class MLPDiffuser(nn.Module):
    def __init__(self, dim=28 * 28, hidden=512):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.Linear(64, hidden), nn.ReLU(), nn.Linear(hidden, dim)
        )
        self.net = nn.Sequential(
            nn.Linear(dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, dim),
        )

    def forward(self, x, t):
        t_emb = self.time_mlp(timestep_embedding(t, 64))
        x = x.view(x.size(0), -1)
        h = self.net(x + t_emb)
        return h.view(x.size(0), 1, 28, 28)


# ---------------- Train ----------------
seed_all(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# Data
tfm = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)
ds = datasets.MNIST("./data", train=True, download=True, transform=tfm)
dl = DataLoader(ds, batch_size=128, shuffle=True)


model = MLPDiffuser().to(device)
diffusion = Diffusion()
opt = optim.AdamW(model.parameters(), lr=2e-4)


os.makedirs("samples", exist_ok=True)


for epoch in range(20):
    for x0, _ in dl:
        x0 = x0.to(device)
        b = x0.shape[0]
        t = torch.randint(0, diffusion.T, (b,), device=device)
        x_t, noise = diffusion.q_sample(x0, t)
        pred = model(x_t, t)
        loss = F.mse_loss(pred, noise)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f"Epoch {epoch}: loss={loss.item():.4f}")

    with torch.no_grad():
        samples = diffusion.sample(model, (64, 1, 28, 28), device)
        samples = (samples.clamp(-1, 1) + 1) / 2
        grid = utils.make_grid(samples, nrow=8)
        utils.save_image(grid, f"samples/epoch_{epoch:02d}.png")
        print("Saved sample image")

Device: cpu
Epoch 0: loss=0.8674
Saved sample image
Epoch 1: loss=0.7998
Saved sample image
Epoch 2: loss=0.7358
Saved sample image
Epoch 3: loss=0.6974
Saved sample image
Epoch 4: loss=0.6907
Saved sample image
Epoch 5: loss=0.6503
Saved sample image
Epoch 6: loss=0.6253
Saved sample image
Epoch 7: loss=0.6644
Saved sample image
Epoch 8: loss=0.6182
Saved sample image
Epoch 9: loss=0.6072
Saved sample image
Epoch 10: loss=0.6349
Saved sample image
Epoch 11: loss=0.6112
Saved sample image
Epoch 12: loss=0.6263
Saved sample image
Epoch 13: loss=0.6189
Saved sample image
Epoch 14: loss=0.6262
Saved sample image
Epoch 15: loss=0.6093
Saved sample image
Epoch 16: loss=0.6021
Saved sample image
Epoch 17: loss=0.6512
Saved sample image
Epoch 18: loss=0.6113
Saved sample image
Epoch 19: loss=0.5935
Saved sample image


In [12]:
with torch.no_grad():
    samples = diffusion.sample(model, (64, 1, 28, 28), device)
    samples = (samples.clamp(-1, 1) + 1) / 2
    grid = utils.make_grid(samples, nrow=8)
    utils.save_image(grid, f"samples/epoch_{epoch:02d}.png")
    print("Saved sample image")

torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64